In [1]:
#Leer los datos

import pandas as pd
from scipy.stats import linregress
import math
from scipy.interpolate import interp1d
from scipy.fft import fft, fftfreq
import numpy as np



lc = pd.read_csv('AP31396872.csv', usecols= ['hjd','mag', 'mag_err'])

In [6]:
#Funciones:

def promedio():
    suma = 0
    for i in lc['mag']:
        suma = suma + i
    return suma/len(lc['mag'])


def mediana():
    #ordenar los datos:
    lista = sorted(lc['mag'])
    cant = len(lista)
    
    #Condiciones
    if cant%2==0:
        mediana = (lista[cant // 2 - 1] + lista[cant // 2]) / 2
    
    else:
        mediana = lista[cant // 2]
    
    return mediana 


def rango():
    lista = sorted(lc['mag'])
    minimo = lista[0]
    maximo = lista[-1]
    
    return maximo-minimo

def percentiles():
    lista = sorted(lc['mag'])
    
    p20_index =  int(0.2 * (len(lista) + 1)) - 1
    p20 = lista[p20_index]
    
    p35_index = int(0.35* (len(lista) + 1)) - 1
    p35 = lista[p35_index]
    
    p50_index = int(0.5* (len(lista) + 1)) - 1
    p50 = lista[p50_index]
    
    p65_index = int(0.65* (len(lista) + 1)) - 1
    p65 = lista[p65_index]
    
    p80_index = int(0.8* (len(lista) + 1)) - 1
    p80 = lista[p80_index]
    
    return p20, p35, p50, p65, p80

mediana = mediana()
promedio = promedio()
    
def amplitud(mediana):
    lista = sorted(lc['mag'])
    amplitud = lista[-1] - mediana
    return amplitud


def varianza(promedio):
    suma = 0
    for i in lc['mag']:
        cantidad = (i -promedio)**2
        suma += cantidad
    return suma/len(lc['mag'])

def kurtosis(promedio):
    suma = 0
    suma2 = 0
    
    N =  len(lc['mag'])
    
    for i in lc['mag']:
        cantidad = (i -promedio)**4
        suma += cantidad
        
        cantidad2 = (i -promedio)**2
        suma2 += cantidad2
        
        
    kurtosis = ((1/N)*suma)/ ((suma2/N)**2) -3
        
    return kurtosis


def autocorrelacion(promedio, lag=1):
    lista = lc['mag']
    n = len(lista)
    
    suma = 0
    suma2 = 0
    
    for i in range(n - lag):
        cantidad = (lista[i] - promedio) * (lista[i + lag] - promedio)
        suma += cantidad
        
        cantidad2 = (lista[i] - promedio) ** 2
        suma2 += cantidad2
    
    autocorrelacion = suma / suma2
        
    return autocorrelacion

def asimetria(promedio):
    
    lista = lc['mag']
    n = len(lista)
    
    suma = 0
    suma2 = 0
    
    for i in range(0, n-1):
        cantidad = (lista[i] - promedio)**3
        suma += cantidad
        
        cantidad2 = (lista[i] - promedio) ** 2
        suma2 += cantidad2
    
    asimetria = (suma/n)/((suma2/n)**(3/2))
        
    return asimetria
    
    

varianza = varianza(promedio)
    
def desviacion_est(varianza):
    desviacion = varianza**(1/2)
    return desviacion 

def pendiente_al():
    x = lc['mag']
    y = lc['hjd']
    
    m, n, r_value, p_value, std_err = linregress(x, y)
    
    return m
def frecuencias_armonicos():
    """
    Calcula las frecuencias de los armónicos.

    Parámetros:
    - tiempo: 
        Array que representa el tiempo.
    - num_armonicos: int, opcional
        Número máximo de armónicos a incluir.

    Retorna:
    - frecuencias_armonicos: np.array
        Array que contiene las frecuencias de los armónicos.
    """
    
    tiempo = lc['hjd']
    num_armonicos = 4
    
    fs = 1 / (tiempo[1] - tiempo[0])

    frequencies = fftfreq(len(tiempo), 1/fs)

    positive_frequencies = frequencies[:len(frequencies)//2]

 
    frecuencias_armonicos = positive_frequencies[1:num_armonicos+1]

    return frecuencias_armonicos


frecuencia_fundamental = frecuencias_armonicos()[0]


def periodo(frecuencia_fundamental):
    
    periodo = 1/frecuencia_fundamental
    
    return periodo

def periodograma():
    
    tiempo = lc['hjd']
    flujo = lc['mag']
    
    N = len(tiempo)
    dt = tiempo[1] - tiempo[0]

    # Calcular las frecuencias
    frecuencias = [1 / (N * dt) * i for i in range(N)]

    # Calcular el periodograma manualmente
    potencia = []
    for f in frecuencias:
        cos_term = sum([fl * math.cos(2 * math.pi * f * t) for t, fl in zip(tiempo, flujo)])
        sin_term = sum([fl * math.sin(2 * math.pi * f * t) for t, fl in zip(tiempo, flujo)])
        potencia.append((cos_term**2 + sin_term**2) / N)

    return frecuencias, potencia



print(periodograma())

([-0.0, -4.964379287868957e-06, -9.928758575737914e-06, -1.489313786360687e-05, -1.9857517151475828e-05, -2.4821896439344785e-05, -2.978627572721374e-05, -3.47506550150827e-05, -3.9715034302951655e-05, -4.467941359082061e-05, -4.964379287868957e-05, -5.4608172166558526e-05, -5.957255145442748e-05, -6.453693074229643e-05, -6.95013100301654e-05, -7.446568931803436e-05, -7.943006860590331e-05, -8.439444789377226e-05, -8.935882718164122e-05, -9.432320646951019e-05, -9.928758575737914e-05, -0.00010425196504524809, -0.00010921634433311705, -0.00011418072362098602, -0.00011914510290885497, -0.00012410948219672392, -0.00012907386148459287, -0.00013403824077246184, -0.0001390026200603308, -0.00014396699934819974, -0.00014893137863606872, -0.00015389575792393767, -0.00015886013721180662, -0.00016382451649967557, -0.00016878889578754452, -0.0001737532750754135, -0.00017871765436328245, -0.0001836820336511514, -0.00018864641293902038, -0.00019361079222688933, -0.00019857517151475828, -0.0002035395